## IMS mismatch with M1M3 motion system - SITCOM-760

In [ ]:
# Define time frame to retrieve data in the EFD database
t_start = "2023-05-30T04:35:00"
t_end = "2023-05-30T05:30:00"

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import itertools as itt
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from astropy import units as u
from astropy.time import Time, TimezoneInfo

from lsst.sitcom import vandv
from lsst.ts.idl.enums import MTM1M3

In [ ]:
# Create client to access the EFD database
client = vandv.efd.create_efd_client()

In [ ]:
all_columns = ["xPosition", "xRotation", "yPosition", "yRotation", "zPosition", "zRotation"]
pos_columns = [c for c in all_columns if "Position" in c]
rot_columns = [c for c in all_columns if "Rotation" in c]

In [ ]:
# get IMS data
df_ims = await client.select_time_series(
    "lsst.sal.MTM1M3.imsData", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_ims = df_ims.set_index("private_rcvStamp")
df_ims.index = pd.to_datetime(df_ims.index, unit="s")

df_ims = df_ims[all_columns]

# Convert meter to milimeter to make is easier to analyse
df_ims[pos_columns] = df_ims[pos_columns] * 1e3

In [ ]:
# get Hard Point state

df_HPState = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_hardpointActuatorState", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_HPState = df_HPState.set_index("private_rcvStamp")
df_HPState.index = pd.to_datetime(df_HPState.index, unit="s")

In [ ]:
# get Hard Point Data
df_HPData = await client.select_time_series(
    "lsst.sal.MTM1M3.hardpointActuatorData", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_HPData = df_HPData.set_index("private_rcvStamp")
df_HPData.index = pd.to_datetime(df_HPData.index, unit="s")

df_HPData = df_HPData[all_columns]

# convert positions to mm
df_HPData[pos_columns] = df_HPData[pos_columns] * 1e3

In [ ]:
# Find all the time windows during which the HP is in Standy state (stable)  
mask_HP_stable = (df_HPState[[f"motionState{j}" for j in range(1, 6)]] == 0).all(axis=1) 

delta_t = pd.Timedelta(0.5, "seconds")
time_stable = []

for i, mask in enumerate(mask_HP_stable):
    if mask:
        if i+1 < len(mask_HP_stable) and not mask_HP_stable[mask_HP_stable.index[i+1]]:
            t0 = mask_HP_stable.index[i]
            
# Remove a small time interval at the end of the time window in order to make sure that HP is in Standy state for any ids timestamp 
            t1 = mask_HP_stable.index[i+1] - delta_t
            time_stable.append([t0, t1])

## Data analysis

### We run the analysis in 2 steps, first the positions and second the rotations

## Position analysis

In [ ]:
# Store results in a dataframe in order to plot them later
results = pd.DataFrame(columns=["ids_x_mean", "ids_y_mean", "ids_z_mean", "HP_x_mean", "HP_y_mean", "HP_z_mean", 
                                "offset_x_mean", "offset_y_mean", "offset_z_mean"])

for index, time_window in enumerate(time_stable):
    t0 = time_window[0]
    t1 = time_window[1]
    ids_x_mean = np.mean(df_ims[t0:t1]['xPosition'])
    ids_y_mean = np.mean(df_ims[t0:t1]['yPosition'])
    ids_z_mean = np.mean(df_ims[t0:t1]['zPosition'])
    HP_x_mean = np.mean(df_HPData[t0:t1]['xPosition'])
    HP_y_mean = np.mean(df_HPData[t0:t1]['yPosition'])
    HP_z_mean = np.mean(df_HPData[t0:t1]['zPosition'])
    offset_x_mean = ids_x_mean - HP_x_mean
    offset_y_mean = ids_y_mean - HP_y_mean
    offset_z_mean = ids_z_mean - HP_z_mean
    
    # Store results in dataframe
    row = pd.DataFrame({"ids_x_mean":ids_x_mean, "ids_y_mean":ids_y_mean, "ids_z_mean":ids_y_mean, 
                        "HP_x_mean":HP_x_mean, "HP_y_mean":HP_y_mean, "HP_z_mean":HP_z_mean,
                        "offset_x_mean":offset_x_mean, "offset_y_mean":offset_y_mean, "offset_z_mean": offset_z_mean}, index=[index])
    results = pd.concat([results, row])

axes = [ "x", "y", "z"]
sigma = {}
mean = {}

for axis in axes:
    sigma[axis] = np.std(results["offset_" + axis + "_mean"])
    mean[axis] = np.mean(results["offset_" + axis + "_mean"])

In [ ]:
results

In [ ]:
# Plot results for position analysis
locator = mdates.AutoDateLocator(minticks=3, maxticks=3)
formatter = mdates.ConciseDateFormatter(locator)

title = "SITCOM-760 - Position Offsets between ims and HP data"
fig, axs = plt.subplots(num=title, nrows=2, ncols=3, figsize=(12, 8))

axes = [ "x", "y", "z"]
for i in range(3):
    max = np.max(results[f"offset_{axes[i]}_mean"])
    min = np.min(results[f"offset_{axes[i]}_mean"])
    axs[0][i].hist(results[f"offset_{axes[i]}_mean"], bins=50, range=[min-0.1*abs(min), max+0.1*abs(max)])
    axs[0][i].set_xlabel(f"Offset (ids-HP) {axes[i]} axis (mm)")
    val_mean = mean[axes[i]]
    val_sigma = sigma[axes[i]]
    axs[0][i].set_title("mean = %.2f mm \n sigma = %.2e mm" %(val_mean, val_sigma))
    
    axs[1][i].plot(df_ims.index, df_ims[axes[i] + "Position"], label="ims")
    axs[1][i].plot(df_HPData.index, df_HPData[axes[i] + "Position"], label = "HP")
    axs[1][i].xaxis.set_major_formatter(formatter)
    max = np.max([np.max(df_ims[axes[i] + "Position"]), np.max(df_HPData[axes[i] + "Position"])])
    min = np.min([np.min(df_ims[axes[i] + "Position"]), np.min(df_HPData[axes[i] + "Position"])])
    axs[1][i].set_ylim([min-0.2, max+0.2])
    axs[1][i].set_xlabel("Time")
    axs[1][i].set_ylabel(f"{axes[i]} Position (mm)")
    axs[1][i].legend()
    
fig.suptitle(title + "\n")
fig.tight_layout()
plt.subplots_adjust(top=0.90)

## Rotations

In [ ]:
# Store results in a dataframe in order to plot them later
results_rot = pd.DataFrame(columns=["ids_xRot_mean", "ids_yRot_mean", "ids_zRot_mean", "HP_xRot_mean", "HP_yRot_mean", "HP_zRot_mean", 
                                "offset_xRot_mean", "offset_yRot_mean", "offset_zRot_mean"])

for index, time_window in enumerate(time_stable):
    t0 = time_window[0]
    t1 = time_window[1]
    ids_xRot_mean = np.mean(df_ims[t0:t1]['xRotation'])
    ids_yRot_mean = np.mean(df_ims[t0:t1]['yRotation'])
    ids_zRot_mean = np.mean(df_ims[t0:t1]['zRotation'])
    HP_xRot_mean = np.mean(df_HPData[t0:t1]['xRotation'])
    HP_yRot_mean = np.mean(df_HPData[t0:t1]['yRotation'])
    HP_zRot_mean = np.mean(df_HPData[t0:t1]['zRotation'])
    offset_xRot_mean = ids_xRot_mean - HP_xRot_mean
    offset_yRot_mean = ids_yRot_mean - HP_yRot_mean
    offset_zRot_mean = ids_zRot_mean - HP_zRot_mean
    
    # Store results in dataframe
    row = pd.DataFrame({"ids_xRot_mean":ids_xRot_mean, "ids_yRot_mean":ids_yRot_mean, "ids_zRot_mean":ids_yRot_mean, 
                        "HP_xRot_mean":HP_xRot_mean, "HP_yRot_mean":HP_yRot_mean, "HP_zRot_mean":HP_zRot_mean,
                        "offset_xRot_mean":offset_xRot_mean, "offset_yRot_mean":offset_yRot_mean, "offset_zRot_mean": offset_zRot_mean}, index=[index])
    results_rot = pd.concat([results_rot, row])

axes = [ "x", "y", "z"]
sigma_rot = {}
mean_rot = {}

for axis in axes:
    sigma_rot[axis] = np.std(results_rot["offset_" + axis + "Rot_mean"])
    mean_rot[axis] = np.mean(results_rot["offset_" + axis + "Rot_mean"])

In [ ]:
results_rot

In [ ]:
# Plot results for rotation analysis
locator = mdates.AutoDateLocator(minticks=3, maxticks=3)
formatter = mdates.ConciseDateFormatter(locator)

title = "SITCOM-760 Rotations Offsets between ims and HP data"
fig, axs = plt.subplots(num=title, nrows=2, ncols=3, figsize=(12, 8))

axes = [ "x", "y", "z"]
for i in range(3):
    max = np.max(results_rot[f"offset_{axes[i]}Rot_mean"])
    min = np.min(results_rot[f"offset_{axes[i]}Rot_mean"])
    axs[0][i].hist(results_rot[f"offset_{axes[i]}Rot_mean"], bins=50, range=[min-0.1*abs(min), max+0.1*abs(max)])
    axs[0][i].set_xlabel(f"Offset (ids-HP) {axes[i]} Rotation (deg)")
    val_mean_rot = mean_rot[axes[i]]
    val_sigma_rot = sigma_rot[axes[i]]
    axs[0][i].set_title("mean = %.4f deg \n sigma = %.e deg" %(val_mean_rot, val_sigma_rot))
    
    axs[1][i].plot(df_ims.index, df_ims[axes[i] + "Rotation"], label="ims")
    axs[1][i].plot(df_HPData.index, df_HPData[axes[i] + "Rotation"], label = "HP")
    axs[1][i].xaxis.set_major_formatter(formatter)
    max = np.max([np.max(df_ims[axes[i] + "Rotation"]), np.max(df_HPData[axes[i] + "Rotation"])])
    min = np.min([np.min(df_ims[axes[i] + "Rotation"]), np.min(df_HPData[axes[i] + "Rotation"])])
    axs[1][i].set_ylim([min-0.015, max+0.015])
    axs[1][i].set_xlabel("Time")
    axs[1][i].set_ylabel(f"{axes[i]} Rotation (deg)")
    axs[1][i].legend()
    
fig.suptitle(title + "\n")
fig.tight_layout()
plt.subplots_adjust(top=0.90)